In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re
import warnings
warnings.filterwarnings('ignore')
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import json
from datetime import datetime, timedelta
import os
chromedriver = "/Users/stevenwang/Projects/funded-and-hiring-site/newsletter_scraper/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
import pyperclip
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pytest_timeout
import dateparser

In [ ]:
def get_content(url):
    page = requests.get(url)
    return page.content

In [ ]:
ttnews_content = get_content('https://www.ttnews.com/articles')

In [ ]:
fw_news_content = get_content('https://www.freightwaves.com/news/category/trucking')

In [94]:
overdrive_business_content = get_content('https://www.overdriveonline.com/business')

In [106]:
samsara_news_content = get_content('https://www.samsara.com/company/news')

In [126]:
def get_ttnews(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='views-row')
    
    for article in articles:
        title = article.find('a')
        date = article.find(class_='date-display-single')
        text =  article.find('p') # description
        image = article.find('img')

        if (date != None and title != None and text != None):
            parsed_date = dateparser.parse(date['content'])
            companies.append('TTNews')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append('https://ttnews.com' + title.get('href'))
            titles.append(title.getText().strip())
            images.append(image.get('src'))
            company_urls.append('https://ttnews.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

ttnews = get_ttnews(ttnews_content)

ttnews

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,"No Vote Held on Infrastructure, but Talks to R...",WASHINGTON — Despite a long night of frantic n...,2021-10-01,https://ttnews.com/articles/no-vote-held-infra...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
1,TTNews,TT Newsmakers,"Sgt. Zach Heard, Illinois State Police, and Ke...",2021-09-30,https://ttnews.com/newsmakers,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
2,TTNews,ATRI Report Urges Electric Tax to Help Fund Hi...,A new report from the American Transportation ...,2021-09-30,https://ttnews.com/articles/atri-report-urges-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
3,TTNews,Transportation Stakeholders Press Congress on ...,To guarantee long-term funding for the country...,2021-09-30,https://ttnews.com/articles/transportation-sta...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
4,TTNews,Congress Clears Government Funding Bill to Ave...,The House passed a nine-week spending bill to ...,2021-09-30,https://ttnews.com/articles/senate-passes-fund...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
5,TTNews,Transport Topics Radio,Transport Topics Radio is a weekly news roundt...,2021-09-30,https://ttnews.com/transport-topics-radio,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
6,TTNews,Port Leaders Warn of Supply Chain Challenges a...,"AUSTIN, Texas — The leader of one of the natio...",2021-09-30,https://ttnews.com/articles/port-leaders-warn-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
7,TTNews,Transportation Stakeholders Focus on Potential...,Federal lawmakers are working on numerous regu...,2021-09-30,https://ttnews.com/articles/transportation-sta...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
8,TTNews,FMCSA Advised to Give Drivers 90 Days to Repla...,A medical advisory board to the Federal Motor ...,2021-09-30,https://ttnews.com/articles/fmcsa-advised-give...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
9,TTNews,Lordstown to Sell Ohio Plant to Foxconn in $28...,Lordstown Motors Corp. agreed to partner with ...,2021-09-30,https://ttnews.com/articles/lordstown-nears-de...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...


In [121]:
def get_overdrive_business(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='node-list__node')
    # print(articles)
    for article in articles:
        title = article.find('h5', class_='section-feed-content-node__content-short-name').a
        # print(title)
        date = article.find(class_='section-feed-content-node__content-published')
        # print(date)
        text =  article.find(class_='section-feed-content-node__content-teaser').a
        # print(text)
        image = article.find('img')

        if (date != None and title != None and text != None):
            parsed_date = dateparser.parse(date.get_text())
            companies.append('Overdrive')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append('https://www.overdriveonline.com' + title.get('href'))
            titles.append(title.getText().strip())
            images.append(image['src'])
            company_urls.append('https://www.overdriveonline.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

overdrive_business = get_overdrive_business(overdrive_business_content)

overdrive_business

,company,headline,description,publish_date,story_url,company_url,image
0,Overdrive,An edge on growth: One way small fleets stand ...,Could Biden's vax mandate enhance the competit...,2021-09-30,https://www.overdriveonline.com/channel-19/art...,https://www.overdriveonline.com/,https://img.overdriveonline.com/files/base/ran...
1,Overdrive,Landstar owner-op poised for growth after winn...,Texas-based hauler Mike Lamb said winning the ...,2021-09-30,https://www.overdriveonline.com/life/article/1...,https://www.overdriveonline.com/,https://img.overdriveonline.com/files/base/ran...
2,Overdrive,Bryan Smith bags the big prize from TCA/Overdrive,The Owner-Operator of the Year was named Tuesd...,2021-09-29,https://www.overdriveonline.com/owner-operator...,https://www.overdriveonline.com/,https://img.overdriveonline.com/files/base/ran...
3,Overdrive,Partners in Business tip: Work to improve your...,Do not wait around and accumulate more debt ov...,2020-12-02,https://www.overdriveonline.com/partners-in-bu...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
4,Overdrive,Fuel prices surge to nearly 7-year high,The modest price increase over the past week b...,2021-09-28,https://www.overdriveonline.com/business/artic...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
5,Overdrive,Are owner-operators a legal landmine for fleet...,Recent court rulings cast doubt on the future ...,2021-09-28,https://www.overdriveonline.com/home/article/1...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
6,Overdrive,FMCSA shuts down New York-based trucker after ...,Bobir M. Kholmurodov has been effectively shut...,2021-09-27,https://www.overdriveonline.com/regulations/ar...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
7,Overdrive,Breaking Free: The history of the owner-operat...,From fighting for freedom to haul in its early...,2011-09-01,https://www.overdriveonline.com/business/artic...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
8,Overdrive,Tractor-trailers banned from section of I-95; ...,DelDot urges truckers to use I-495 as an alter...,2021-09-23,https://www.overdriveonline.com/home/article/1...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
9,Overdrive,DOT asks for truckers' input on supply chain d...,DOT will use information from the trucking ind...,2021-09-22,https://www.overdriveonline.com/regulations/ar...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."


In [122]:
def get_samsara_news(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='row')
    # print(articles)
    for article in articles:
        title = article.find(class_='source-quote')
        link = article.find('a')
        # print(title)
        date = article.find(class_='source-date')
        # print(date)
        text =  article.find(class_='source-quote')
        # print(text)
        image = article.find('img')
        # print(image)

        if (link != None and date != None and title != None and text != None):
            parsed_date = dateparser.parse(date.get_text())
            companies.append('Samsara')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append(link.get('href'))
            titles.append(title.getText().strip())
            
            if (image != None and image.get('src')):
                images.append(image['src'])
            elif (image != None and image.get('data-lazy')):
                images.append(image['data-lazy'])
            else:
                images.append(None)
            
            company_urls.append('https://www.samsara.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

samsara_news = get_samsara_news(samsara_news_content)

samsara_news

,company,headline,description,publish_date,story_url,company_url,image
0,Samsara,"""Samsara Ranked on LinkedIn's 2021 Top Startup...","""Samsara Ranked on LinkedIn's 2021 Top Startup...",2021-09-22,https://www.prnewswire.com/news-releases/samsa...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/6Ey6tmvh1M...
1,Samsara,"""The State of Tennessee has awarded Samsara a ...","""The State of Tennessee has awarded Samsara a ...",2021-09-17,https://www.government-fleet.com/10151760/tenn...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/47F6OkjxOk...
2,Samsara,"""Samsara Selected as Connected Operations Plat...","""Samsara Selected as Connected Operations Plat...",2021-09-13,https://www.prnewswire.com/news-releases/samsa...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/2o8KuZK17L...
3,Samsara,"""Samsara Announces Confidential Submission of ...","""Samsara Announces Confidential Submission of ...",2021-09-01,https://www.prnewswire.com/news-releases/samsa...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/66jIszSDer...
4,Samsara,"""Ethical AI is still in its nascency but is be...","""Ethical AI is still in its nascency but is be...",2021-08-29,https://venturebeat.com/2021/08/29/how-to-make...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/2UpvaHxdNk...
...,...,...,...,...,...,...,...
95,Samsara,"""Some telling data points come this week from ...","""Some telling data points come this week from ...",2020-06-16,https://mashable.com/article/speeding-crashes-...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/7FJJ0zDU2t...
96,Samsara,"""While speeds were rising, incidents of harsh ...","""While speeds were rising, incidents of harsh ...",2020-06-16,https://finance.yahoo.com/news/truck-traffic-n...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/5nqMKhHlvz...
97,Samsara,"""To help maintain social distancing protocols,...","""To help maintain social distancing protocols,...",2020-06-15,https://www.automotive-fleet.com/359959/8-ways...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/2khYrATrnU...
98,Samsara,"""The newest member of Together for Safer Roads...","""The newest member of Together for Safer Roads...",2020-06-11,https://www.freightwaves.com/news/commentary-i...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/5NgX9gwLrd...


In [123]:
def get_freightWaves(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []

    articles = soup.find_all(class_='post-item')
    
    for article in articles:
        title = article.find('a')
        date =  article.find(class_='date meta-item tie-icon')
        text =  article.find('p')
        image = article.find('img')

        companies.append('FreightWaves')
        description.append(text.getText())
        publish_date.append(dateparser.parse(date.get_text()).date())
        stories.append(title.get('href'))
        titles.append(title.get('aria-label'))
        images.append(image)
        company_urls.append('https://www.freightwaves.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe
        
fw_news = get_freightWaves(fw_news_content)

fw_news

,company,headline,description,publish_date,story_url,company_url,image
0,FreightWaves,Lordstown Motors may sell its plant to Taiwan’...,Taiwan’s Foxconn would manufacture Lordstown M...,2021-09-30,https://www.freightwaves.com/news/lordstown-mo...,https://www.freightwaves.com/,[]
1,FreightWaves,Ransomware attack on Forward Air may have expo...,Nine months after a ransomware attack on Forwa...,2021-09-30,https://www.freightwaves.com/news/ransomware-a...,https://www.freightwaves.com/,[]
2,FreightWaves,Workhorse executive housecleaning continues wi...,"New CEO continues remaking executive team, wit...",2021-09-30,https://www.freightwaves.com/news/workhorse-ex...,https://www.freightwaves.com/,[]
3,FreightWaves,FreightWaves Classics: I-24 travels through fo...,I-24 passes through four states and provides a...,2021-09-30,https://www.freightwaves.com/news/freightwaves...,https://www.freightwaves.com/,[]
4,FreightWaves,M&A activity staying hot across logistics indu...,Pent-up demand is making its way to the surfac...,2021-09-30,https://www.freightwaves.com/news/ma-activity-...,https://www.freightwaves.com/,[]
5,FreightWaves,ArcBest sees billions of dollars in opportunit...,ArcBest’s acquisition of truckload broker MoLo...,2021-09-30,https://www.freightwaves.com/news/arcbest-sees...,https://www.freightwaves.com/,[]
6,FreightWaves,"Be prepared, attorneys say: More driver classi...",A pair of lawyers tell a TCA audience that tak...,2021-09-30,https://www.freightwaves.com/news/be-prepared-...,https://www.freightwaves.com/,[]
7,FreightWaves,Successful driver recruitment is multifaceted ...,Tenstreet partnered with FreightWaves to bette...,2021-09-30,https://www.freightwaves.com/news/successful-d...,https://www.freightwaves.com/,[]
8,FreightWaves,"Pick your bottleneck: Ports, chassis, containe...",With disruptions likely to linger well into 20...,2021-09-30,https://www.freightwaves.com/news/pick-your-bo...,https://www.freightwaves.com/,[]
9,FreightWaves,Report: Electric vehicle tax could revive High...,An ATRI study shows how electric vehicles can ...,2021-09-29,https://www.freightwaves.com/news/report-elect...,https://www.freightwaves.com/,[]


In [127]:
final = pd.concat([ttnews, fw_news, overdrive_business, samsara_news], axis = 0)
#replace all .coms in company names
final['company'] = final.company.str.replace("[.].*|•|’|,|[+]|[()]|[#]", '').str.strip()

In [128]:
final

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,"No Vote Held on Infrastructure, but Talks to R...",WASHINGTON — Despite a long night of frantic n...,2021-10-01,https://ttnews.com/articles/no-vote-held-infra...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
1,TTNews,TT Newsmakers,"Sgt. Zach Heard, Illinois State Police, and Ke...",2021-09-30,https://ttnews.com/newsmakers,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
2,TTNews,ATRI Report Urges Electric Tax to Help Fund Hi...,A new report from the American Transportation ...,2021-09-30,https://ttnews.com/articles/atri-report-urges-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
3,TTNews,Transportation Stakeholders Press Congress on ...,To guarantee long-term funding for the country...,2021-09-30,https://ttnews.com/articles/transportation-sta...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
4,TTNews,Congress Clears Government Funding Bill to Ave...,The House passed a nine-week spending bill to ...,2021-09-30,https://ttnews.com/articles/senate-passes-fund...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
...,...,...,...,...,...,...,...
95,Samsara,"""Some telling data points come this week from ...","""Some telling data points come this week from ...",2020-06-16,https://mashable.com/article/speeding-crashes-...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/7FJJ0zDU2t...
96,Samsara,"""While speeds were rising, incidents of harsh ...","""While speeds were rising, incidents of harsh ...",2020-06-16,https://finance.yahoo.com/news/truck-traffic-n...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/5nqMKhHlvz...
97,Samsara,"""To help maintain social distancing protocols,...","""To help maintain social distancing protocols,...",2020-06-15,https://www.automotive-fleet.com/359959/8-ways...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/2khYrATrnU...
98,Samsara,"""The newest member of Together for Safer Roads...","""The newest member of Together for Safer Roads...",2020-06-11,https://www.freightwaves.com/news/commentary-i...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/5NgX9gwLrd...


In [ ]:
final.head()

In [ ]:
len(final)

In [ ]:
#check previous posts, check to see there are no duplicates
# old = pd.read_csv('historical_deals.csv')
# dupes = pd.merge(final, old[['company', 'deal_size']], how = 'inner', on = ['company', 'deal_size'])
# final = final[~final.company.isin(dupes.company)]

In [ ]:
len(final)

In [ ]:
final

In [129]:
#save deal records
# new_old = pd.concat([final, old], axis = 0)
new_old = pd.concat([final], axis = 0)

new_old.to_csv('historical_deals.csv', index = False)

In [105]:
len(new_old)

48

In [ ]:
date = 'Sunday, 08/29/2021'

title = '''
<h1>
  Funded & Hiring
</h1>
<h2 style="text-align:center">
  <strong>{date}</strong>
</h2>
<h2>
  A weekly roundup of funded startups and the jobs they're hiring for.
</h2>
<h4>
  Forwarded from a friend? 
  <a href = 'https://fundedandhiring.com/'>Subscribe</a> to stay up to date on the latest startup funding and job alerts.
</h4>
'''.format(date = date).strip()

In [ ]:
print(title)
pyperclip.copy(title)

In [ ]:
content = ''''''

final_dict = final.to_dict(orient = 'records')
final_dict = final_dict[0:7]
for row in final_dict:
    headline = row['headline']
    company_name = row['company']
    publish_date = row['publish_date']
    description = row['description']
    story_url = row['story_url']
    company_url = row['company_url']
    img = row['image']

    row_url = company_url + story_url
    
    entry = '''
    <tr>
      <td>
        {img}
        <h3>
          <a href = "{row_url}">{headline}</a>
        </h3>
        <p>
          {description}
        </p>
        <p>
          <a href = "{company_url}">{company_name}</a>
        </p>
        <p>
          <b>{publish_date}</b>
        </p>
      </td>
    </tr> '''.format(row_url = row_url, headline = headline, company_url = company_url, company_name = company_name, description = description, publish_date = publish_date, img=img).strip()

    content += '\n' + entry.strip()

In [ ]:
pyperclip.copy(content)
print(content)

In [ ]:
#job boards
# stackoverflow, jobvite, bamboohr